#### EX 4.4 Serialization and Composition

디스크에서 prompt template를 가져오는 방법에 대해서 학습한다.

1. prompt 형식을 DB에 저장하기 위해 .json, .yaml 확장자로 파일화하여 관리하는 법을 알아본다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# prompt를 별도의 파일로 관리할수 있도록 코드를 정비한다
# .json, .yaml 형식의 확장자를 사용한다. 입력된 내용은 파일 확인요망
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")

c:\Users\smin9\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'What is the capital of Germany'

2. prompt들의 memory등을 모으는 방법에 대해서 알아본다.  
 (다수의 prompt들을 관리하기에 용이하며, 결국엔 하나로 다 합치게 된다.) 


3. 롤플레잉 형식으로 AI에게 역할을 지정해주고 맡은 역할에 어울리는 답변을 하도록 한다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 롤플레잉 할 캐릭터를 지정하면 해당 역할을 연기
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

# 캐릭터가 어떻게 질문, 답변하는지에 대한 예제
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

# AI 도우미가 우리의 텍스트를 완성해주는 예제
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

# prompt를 모두 합친다
final = PromptTemplate.from_template(
    """
    {intro}
                                    
    {example}
                            
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your favorate food?",
    }
)

Arrrrg! Me favorate food be a good ol' plate o' fish 'n' chips! The taste o' crispy battered fish and salty chips be like a treasure for me taste buds, matey! Arg arg!

AIMessageChunk(content="Arrrrg! Me favorate food be a good ol' plate o' fish 'n' chips! The taste o' crispy battered fish and salty chips be like a treasure for me taste buds, matey! Arg arg!")